In [ ]:
dbutils.library.restartPython()

In [ ]:
import os
os.sys.path.append("/Workspace/Repos/delsonscyan@hkjccorpdev.onmicrosoft.com/stretch_goal/src/elson")

## import libs and spark session

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("demo").getOrCreate()

In [ ]:
from data_clean.column_transformer import Cleansing

## create a source dataframe

In [ ]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
schema = StructType([
    StructField("id", StringType(), True),
    StructField("name", StringType(), True),
    StructField("gender", StringType(), True),
    StructField("country", StringType(), True),
    StructField("age", IntegerType(), True),
])
data = [
    ("001","AaaA","male","US",1),
    (""   ,"BBBB","male","UK",2),
    ("003",""    , None , "" ,3),
]
rate_df = spark.createDataFrame(data, schema)
rate_df.display()

## instance a Cleansing obj and add rules

In [ ]:
cleansing = Cleansing(
        rate_df,
        "./test_rules.yaml",
    )

## use builtin rules

In [ ]:
(cleansing
    .add_rule("Rule1")
    .add_column("id", "name", "country")
    .add_rule("null_2_string")
    .add_column("gender")
)

In [ ]:
cleansing.exec().display()

In [ ]:
cleansing.exec().display()

## add rules in same instance obj

In [ ]:
(cleansing
    .add_rule("Rule1")
    .add_column("gender")
    .exec().display()
)

## add a customize rule from local which is not builtin ...

In [ ]:
from data_clean.rules import external_rule
from pyspark.sql import DataFrame, functions as F
from data_clean.cleansing_utils import Origin_Rule


from data_clean.rules.IntRule import Int_Rule
sr1 = Int_Rule()
print([i for i in sr1.__dir__() if not i.startswith('_')])

@external_rule("int")
def int_suffix(df: DataFrame, origin_rule: Origin_Rule, col: str):
    return df.withColumn("new_"+col, F.concat(F.col(col) ,F.lit("a")))

sr2 = Int_Rule()
print([i for i in sr2.__dir__() if not i.startswith('_')])

In [ ]:
(cleansing
    .add_rule("int_suffix")
    .add_column("age")
    .add_rule("upper_string")
    .add_column("new_age")
    .exec().display()
)

In [ ]:
(cleansing
    .exec().display()
)